In [2]:
from google.colab import drive
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
from PIL import Image
import numpy as np
dataset=[]
#train_dataset=[]
#test_dataset=[]
for n in range(1,141):
  try:
    image = Image.open('/content/drive/MyDrive/ColabNotebooks/Dataset/o_{}.png'.format(n)).convert('L')
  except:
    image = Image.open('/content/drive/MyDrive/ColabNotebooks/Dataset/o_{}.jpg'.format(n)).convert('L')
  image = np.array([image])
  image =torch.FloatTensor(image)
  #data=image,torch.FloatTensor([[0,1]])
  data=image,0
  dataset.append(data)
  try:
    image = Image.open('/content/drive/MyDrive/ColabNotebooks/Dataset/x_{}.png'.format(n)).convert('L')
  except:
    image = Image.open('/content/drive/MyDrive/ColabNotebooks/Dataset/x_{}.jpg'.format(n)).convert('L')
  image = np.array([image])
  image=torch.FloatTensor(image)
  #data=image,torch.FloatTensor([[1,0]])
  data=image,1
  dataset.append(data)

In [4]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)


if __name__ == '__main__':

  # Configuration options
  k_folds = 7
  num_epochs = 10
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler)
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=test_subsampler)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch    10: 16.284
Loss after mini-batch    20: 5.018
Loss after mini-batch    30: 11.297
Loss after mini-batch    40: 7.466
Loss after mini-batch    50: 6.622
Loss after mini-batch    60: 5.966
Loss after mini-batch    70: 5.723
Loss after mini-batch    80: 5.298
Loss aft